In [2]:
# region General Imports
import os
import uuid
import shutil
import time
import random
import datetime
import glob
import pickle
import tqdm
import copy
import optuna
import numpy as np
import pandas as pd
import scipy as sp
import matplotlib.pyplot as plt
import seaborn as sns
import mne
from rich import print as rprint
from rich.pretty import pprint as rpprint
from tqdm import tqdm
from itertools import chain
from functools import partial
# endregion General Imports

from model_optim.model_optimizer import ModelOptimizer

# Dataset
from custom_datasets.fatigue_mi import FatigueMI

%load_ext autoreload
%autoreload 3

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
# np.load("./bkup/best_subjects_search_results/subject_6_shallow_conv_net_7b71b3b6-52f7-435c-be85-2cb99e793f98.npy", allow_pickle=True).item().user_attrs['trial_data']

prev_best_full = np.load("./_temp/[1]/292cbc92b8cf46da9986fe7d8447819f/model/study_best_trial.npy", allow_pickle=True).item()
prev_best = prev_best_full.user_attrs['trial_data']

rpprint({
    "sfreq": prev_best_full.params["sfreq"],
    "batch_size": prev_best_full.params["batch_size"],
    "val_acc": np.max(prev_best["val_accuracy"]),
    "test_acc": prev_best["test_accuracy"],
    "channels": prev_best["channels_selected"],
})

{
│   'sfreq': 300,
│   'batch_size': 160,
│   'val_acc': 0.8333333134651184,
│   'test_acc': 0.8636363744735718,
│   'channels': array(['Fz', 'C4', 'Fp2', 'T5', 'O2', 'F7', 'F8', 'A2', 'T6', 'T4'],
│     dtype='<U3')
}

In [4]:
model_optimizer = ModelOptimizer(
    dataset=FatigueMI(),
    model_name="shallow_conv_net"
)

In [5]:
study = model_optimizer.search_best_model(
    subjects=[1],
    # channels=['Fz', 'C4', 'Fp2', 'T5', 'O2', 'F7', 'F8', 'A2', 'T6', 'T4'],
    # sfreq=300,
    # batch_size=160,
    max_iter=50,
    max_epochs=10,
    max_stag_count=25,
    rounds=1,
    replace_previous_study_for_subjects=True
)

  0%|          | 0/50 [00:00<?, ?it/s]

Adding metadata with 3 columns


2024-03-06 23:26:29.528736: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-06 23:26:29.550415: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-06 23:26:29.550505: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-06 23:26:29.553445: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-06 23:26:29.553578: I external/local_xla/xla/stream_executor

Epoch 1/10


2024-03-06 23:26:31.976788: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inmodel/dropout/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer
2024-03-06 23:26:33.091081: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8907
2024-03-06 23:26:33.895712: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-03-06 23:26:34.574034: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-03-06 23:26:35.682303: I external/local_xla/xla/service/service.cc:168] XLA service 0x7fb7e7a15480 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-03-06 23:26:35.682346: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce GTX 1070, C

...

Best trial: 32. Best value: 3.01721:  96%|█████████▌| 48/50 [02:51<00:06,  3.22s/it]



Sampling frequency of the instance is already 300.0, returning unmodified.
Adding metadata with 3 columns
Epoch 1/10
1/1 [==============================] - 2s 2s/step - loss: 18.3984 - accuracy: 0.5147 - val_loss: 15.4299 - val_accuracy: 0.5556 - lr: 0.0010
Epoch 2/10
1/1 [==============================] - 0s 53ms/step - loss: 15.3478 - accuracy: 0.6912 - val_loss: 14.9587 - val_accuracy: 0.5556 - lr: 0.0010
Epoch 3/10
1/1 [==============================] - 0s 53ms/step - loss: 14.7968 - accuracy: 0.7647 - val_loss: 14.4739 - val_accuracy: 0.5556 - lr: 0.0010
Epoch 4/10
1/1 [==============================] - 0s 52ms/step - loss: 14.3092 - accuracy: 0.8088 - val_loss: 13.9929 - val_accuracy: 0.5556 - lr: 0.0010
Epoch 5/10
1/1 [==============================] - 0s 54ms/step - loss: 13.7784 - accuracy: 0.8382 - val_loss: 13.5202 - val_accuracy: 0.6111 - lr: 0.0010
Epoch 6/10
1/1 [==============================] - 0s 53ms/step - loss: 13.2918 - accuracy: 0.8529 - val_loss: 13.0588 - val_

Best trial: 32. Best value: 3.01721:  98%|█████████▊| 49/50 [02:54<00:03,  3.10s/it]



Adding metadata with 3 columns
Epoch 1/10
1/1 [==============================] - 2s 2s/step - loss: 7.5055 - accuracy: 0.4706 - val_loss: 5.5671 - val_accuracy: 0.3333 - lr: 0.0010
Epoch 2/10
1/1 [==============================] - 0s 79ms/step - loss: 5.4490 - accuracy: 0.6618 - val_loss: 5.4553 - val_accuracy: 0.4444 - lr: 0.0010
Epoch 3/10
1/1 [==============================] - 0s 80ms/step - loss: 5.1727 - accuracy: 0.9118 - val_loss: 5.3298 - val_accuracy: 0.4444 - lr: 0.0010
Epoch 4/10
1/1 [==============================] - 0s 80ms/step - loss: 4.9706 - accuracy: 0.9265 - val_loss: 5.1791 - val_accuracy: 0.5000 - lr: 0.0010
Epoch 5/10
1/1 [==============================] - 0s 79ms/step - loss: 4.7834 - accuracy: 0.9412 - val_loss: 5.0069 - val_accuracy: 0.5000 - lr: 0.0010
Epoch 6/10
1/1 [==============================] - 0s 81ms/step - loss: 4.5810 - accuracy: 0.9853 - val_loss: 4.8246 - val_accuracy: 0.4444 - lr: 0.0010
Epoch 7/10
1/1 [==============================] - 0s 80ms

Best trial: 32. Best value: 3.01721: 100%|██████████| 50/50 [02:58<00:00,  3.56s/it]

In [6]:
model_optimizer.get_study_metrics(study).sort_values(by="scores", ascending=True)
# model_optimizer.get_study_metrics(study).sort_values(by="test_acc", ascending=False)

,train_acc,test_acc,val_acc,train_val_acc_diff,train_loss,val_loss,train_val_loss_diff,test_loss,scores,channels_selected,sfreq,batch_size
32,0.867647,0.409091,0.444444,0.423203,1.704391,1.958066,0.253674,1.993756,3.017207,"[C3, C4, Cz, Pz, Fp2, T5, O1, O2, T6, T4]",256,128
31,0.941176,0.454545,0.555556,0.385621,2.395677,2.715051,0.319374,2.648694,3.913032,"[F3, Fz, C4, P4, Cz, T3, F7, A2, T4]",256,128
49,1.000000,0.590909,0.611111,0.388889,3.820418,4.116434,0.296016,4.149892,4.268369,"[C3, F3, Fz, C4, P4, Cz, Pz, T3, T5, O1, F7, F...",256,192
22,0.735294,0.590909,0.555556,0.179739,3.934094,3.933653,0.000441,3.959220,5.131884,"[C3, Fz, F4, C4, P4, Cz, Pz, Fp2, T3, T5, F7, ...",300,160
38,1.000000,0.500000,0.555556,0.444444,4.710239,4.994878,0.284639,4.985184,5.193059,"[P3, Fz, F4, C4, P4, Cz, Pz, Fp1, O1, F7, A2, ...",300,192
10,0.867647,0.590909,0.722222,0.145425,5.514791,5.550944,0.036152,5.545656,5.628604,"[P3, F3, Fz, F4, Cz, Fp1, T5, O2, T6, T4]",256,192
46,0.750000,0.409091,0.722222,0.027778,4.850857,4.787607,0.063250,4.809487,5.865368,"[P3, F3, C4, P4, Pz, T5, O1, F7, F8, A2, T6, T4]",128,128
34,0.808824,0.409091,0.666667,0.142157,5.819441,5.782907,0.036534,5.836151,5.894519,"[P3, Fz, F4, C4, P4, Cz, Pz, Fp2, O1, A2]",300,128
25,0.808824,0.454545,0.444444,0.364379,5.000159,5.087465,0.087306,5.034462,6.146457,"[P3, C3, F4, P4, Pz, T3, O2]",300,160
43,0.926471,0.727273,0.555556,0.370915,5.870110,6.020370,0.150261,5.880886,6.218451,"[C3, F3, Fz, F4, C4, P4, Fp2, O2, F8, A2, T6]",128,128


In [7]:
rpprint({ k: v for k, v in study.best_trial.params.items() if not k.startswith("channels") })
rprint("test_accuracy =", study.best_trial.user_attrs["trial_data"]["test_accuracy"])
rprint("val_accuracy =", np.max(study.best_trial.user_attrs["trial_data"]["val_accuracy"]))
rprint("channels_selected =", study.best_trial.user_attrs["trial_data"]["channels_selected"])

{
│   'sfreq': 256,
│   'batch_size': 128,
│   'pool_size_d2': 70,
│   'strides_d2': 28,
│   'conv_filters_d2': 35,
│   'conv2d_1_units': 80,
│   'conv2d_2_units': 20,
│   'l2_reg_1': 0.3769065669173121,
│   'l2_reg_2': 0.015620232595905444,
│   'l2_reg_3': 0.28045184269446194,
│   'dropout_rate': 0.7000000000000001
}

test_accuracy = 0.40909090638160706

val_accuracy = 0.4444444477558136

channels_selected = ['C3' 'C4' 'Cz' 'Pz' 'Fp2' 'T5' 'O1' 'O2' 'T6' 'T4']

In [8]:
model_optimizer.clean_up(
    action="remove_all_but_best_trial_data",
    subjects=[1],
)

['./temp/[1]/d7d4eed32ef84bea88b8085d7be995e3/model/study_best_trial.npy']

Found 1 best trials for subjects: [1]